In [28]:
import utilities, myanimelistAPI, jikan4pyAPI, json
import requests as r
import datetime as dt

In [29]:
malID_to_aniListID = {}

In [30]:
def malIDtoAniListID(anime_id):
	"""
	Converts the MAL ID to AniList ID
	using GraphQL
	"""
	if anime_id in malID_to_aniListID:
		return malID_to_aniListID[anime_id]

	query = """query
		($id: Int, $type: MediaType) {
			Media(idMal: $id, type: $type) {
				id}
			}
		"""
	variables = {'id': anime_id, 'type': 'ANIME'}
	url = 'https://graphql.anilist.co'
	response = r.post(url, json={'query': query, 'variables': variables})
	malID_to_aniListID[anime_id] = int(response.json()['data']['Media']['id'])
	return malID_to_aniListID[anime_id]

def timeTillNextEpisode(anime_id):
	"""
	Returns the time till the next episode of the anime airs
	"""
	query = """query ($id: Int) {
					Media(idMal: $id, type: ANIME) {
						id
						nextAiringEpisode {
						timeUntilAiring
						}
					}
				}
	"""
	variables = {'id': anime_id, 'type': 'ANIME'}
	url = 'https://graphql.anilist.co'
	data = r.post(url, json={'query': query, 'variables': variables}).json()

	try: # if parsing fails, then the anime is not airing
		time_data = data['data']['Media']['nextAiringEpisode']
	except:
		time_data = None

	if time_data is not None:
		seconds = time_data['timeUntilAiring']
	else:
		return {"days": 0,
				"hours": 0,
				"minutes": 0,
				"seconds": 0,
				"total_seconds": 0,
				"isNull": True}

	time = dt.timedelta(seconds=seconds)
	return {"days": time.days,
			"hours": time.seconds//3600,
			"minutes": (time.seconds//60) % 60,
			"seconds": time.seconds % 60,
			"total_seconds": seconds,
			"isNull": False}
 
def isAnimeAiring(anime_id, api="mal"):
	"""
	Checks if the anime is airing
	using the MyAnimeList API (atleast more updated then the Jikan API due to less caching)
	"""
	if api == "mal":
		try:
			result = mal_api.getAnimeByID(anime_id)
			return (result['status'] != "finished_airing")
		except Exception as e:
			print_bot(
				f"Error: Mal API ID check failed, so switching to Jikan API.\n'{e}'")
			result = jikan_api.getAnimeByID(anime_id)
			return result["status"] != "Finished Airing"
	else:
		result = jikan_api.getAnimeByID(anime_id)
		return result["status"] != "Finished Airing"

In [36]:
malapi = myanimelistAPI.MyAnimeListAPI(open("secrets.key").read())
jikanAPI = jikan4pyAPI.JikanAPI()

In [37]:
timeTillNextEpisode(jikanAPI.getAnimeIDByName("The eminence in shadow 2"))

{'days': 5,
 'hours': 2,
 'minutes': 31,
 'seconds': 5,
 'total_seconds': 441065,
 'isNull': False}